# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waingapu,-9.6567,120.2641,24.46,81,83,4.20,ID,1688902409
1,1,port-aux-francais,-49.3500,70.2167,1.24,55,91,8.14,TF,1688902409
2,2,waitangi,-43.9535,-176.5597,10.46,84,100,4.47,NZ,1688902409
3,3,grytviken,-54.2811,-36.5092,-5.56,83,42,5.01,GS,1688902409
4,4,edinburgh of the seven seas,-37.0676,-12.3116,15.34,79,100,10.04,SH,1688902409


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] >= 21)
                            & (city_data_df["Cloudiness"] ==0) & (city_data_df["Wind Speed"] <4.50)]

# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df.set_index('City_ID')


/var/folders/d4/c5tpxrvs7cxfgsj80xn7v0600000gn/T/ipykernel_44664/1787130244.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df.dropna(inplace=True)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
22,menongue,-14.6585,17.6910,26.51,15,0,2.71,AO,1688902412
75,kafue,-15.7691,28.1814,25.94,22,0,2.50,ZM,1688902420
175,roma,43.2128,-75.4557,21.51,94,0,2.57,US,1688902436
179,vila velha,-20.3297,-40.2925,21.64,88,0,1.03,BR,1688902436
214,santana,-0.0583,-51.1817,26.10,94,0,2.06,BR,1688902442
342,kermit,31.8576,-103.0927,26.91,54,0,2.57,US,1688902462
456,sunnyside,46.3237,-120.0087,22.69,81,0,0.00,US,1688902480
481,horsham,51.0640,-0.3272,22.63,66,0,3.09,GB,1688902484
492,greenville,35.6127,-77.3663,24.01,100,0,0.00,US,1688902485


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Lat", "Lng", "Humidity", "Country"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name","")

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
22,menongue,-14.6585,17.6910,15,AO,
75,kafue,-15.7691,28.1814,22,ZM,
175,roma,43.2128,-75.4557,94,US,
179,vila velha,-20.3297,-40.2925,88,BR,
214,santana,-0.0583,-51.1817,94,BR,
342,kermit,31.8576,-103.0927,54,US,
456,sunnyside,46.3237,-120.0087,81,US,
481,horsham,51.0640,-0.3272,66,GB,
492,greenville,35.6127,-77.3663,100,US,
554,caconda,-13.7333,15.0667,18,AO,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "limit":10,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
menongue - nearest hotel: No hotel found
kafue - nearest hotel: No hotel found
roma - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
santana - nearest hotel: No hotel found
kermit - nearest hotel: No hotel found
sunnyside - nearest hotel: No hotel found
horsham - nearest hotel: No hotel found
greenville - nearest hotel: No hotel found
caconda - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
22,menongue,-14.6585,17.6910,15,AO,No hotel found
75,kafue,-15.7691,28.1814,22,ZM,No hotel found
175,roma,43.2128,-75.4557,94,US,No hotel found
179,vila velha,-20.3297,-40.2925,88,BR,No hotel found
214,santana,-0.0583,-51.1817,94,BR,No hotel found
342,kermit,31.8576,-103.0927,54,US,No hotel found
456,sunnyside,46.3237,-120.0087,81,US,No hotel found
481,horsham,51.0640,-0.3272,66,GB,No hotel found
492,greenville,35.6127,-77.3663,100,US,No hotel found
554,caconda,-13.7333,15.0667,18,AO,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Lat", "Lng", "Humidity","Country","Hotel Name"],
    geo=True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)